<a href="https://colab.research.google.com/github/bindu972/batch-07/blob/main/Autonomous_Legal_Research_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
!pip install openai langchain pypdf gradio faiss-cpu langchain-community --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.5 MB/s eta 0:00:00


In [21]:
from langchain_community.document_loaders import PyPDFLoader

In [22]:
import openai
import gradio as gr
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
import os

In [23]:
openai.api_key = "YOUR_OPENAI_API_KEY"

In [24]:
def parse_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_documents(documents)
    return chunks

In [25]:
def create_vector_db(chunks):
    embeddings = OpenAIEmbeddings()
    db = FAISS.from_documents(chunks, embeddings)
    return db

In [26]:
def detect_legal_domain(query):
    query = query.lower()
    if "contract" in query:
        return "Contract Law"
    elif "criminal" in query:
        return "Criminal Law"
    elif "intellectual" in query or "patent" in query or "ip" in query:
        return "Intellectual Property Law"
    else:
        return "General Law"

In [28]:
def answer_legal_query(pdf_file, query):
    if pdf_file is None:
        return "Please upload a legal PDF first."

    # Save the uploaded file temporarily
    file_path = pdf_file.name

    # Parse the PDF into chunks
    chunks = parse_pdf(file_path)

    # Create a vector database from the chunks
    vector_db = create_vector_db(chunks)

    # Initialize the language model
    llm = ChatOpenAI(model_name="gpt-4", temperature=0)

    # Create a RetrievalQA chain
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=vector_db.as_retriever()
    )

    # Get the answer to the query
    response = qa_chain.run(query)

    # You might want to delete the temporary file after processing
    # os.remove(file_path)

    return response

In [29]:
gr.Interface(
    fn=answer_legal_query,
    inputs=[
        gr.File(label="📄 Upload Legal PDF"),
        gr.Textbox(label="❓ Ask a legal question (e.g., case summary, related precedents)")
    ],
    outputs=gr.Textbox(label="📚 AI Legal Assistant Response", lines=15),
    title="⚖️ Autonomous Legal Research Assistant",
    description="Upload legal documents (PDFs) and ask natural language queries. AI will extract, analyze, and summarize legal content using GPT-4 + LangChain."
).launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://02c7ea134c3aa36c43.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
